In [37]:
import pandas as pd

## Import Raw Dataset Shop

In [38]:
raw_data=pd.read_csv('dataset_shop.csv')
raw_data

,tid,items
0,1,"Kemeja,Kaos"
1,2,Jeans
2,3,"Jeans,Sendal"
3,4,"Sepatu,Jeans"
4,5,Sepatu
5,6,"Kemeja,Sendal,Kaos"
6,7,"Kaos,Jeans,Kemeja"
7,8,"Sepatu,Kaos,Kemeja"
8,9,"Kaos,Jeans,Kemeja,Sendal"
9,10,"Sepatu,Sendal,Kaos,Kemeja"


In [39]:
cm_seperate=raw_data['items'].apply(lambda x: x.split(','))
cm_seperate

0                     [Kemeja, Kaos]
1                            [Jeans]
2                    [Jeans, Sendal]
3                    [Sepatu, Jeans]
4                           [Sepatu]
5             [Kemeja, Sendal, Kaos]
6              [Kaos, Jeans, Kemeja]
7             [Sepatu, Kaos, Kemeja]
8      [Kaos, Jeans, Kemeja, Sendal]
9     [Sepatu, Sendal, Kaos, Kemeja]
10                   [Kemeja, Jeans]
11                  [Sendal, Kemeja]
12                          [Sendal]
13                   [Sendal, Jeans]
14           [Sepatu, Jeans, Kemeja]
Name: items, dtype: object

# Convert String to Number

In [40]:

item_name = ['Kemeja','Kaos','Jeans','Sepatu','Sendal']
item_no = [1, 2, 3, 4, 5]
recodes=dict(zip(item_name, item_no))
recodes

{'Kemeja': 1, 'Kaos': 2, 'Jeans': 3, 'Sepatu': 4, 'Sendal': 5}

In [41]:
listNumberedItems=[]
for i in range(len(cm_seperate)):
    numbered_list=list(map(lambda x:recodes[x], cm_seperate[i]))
    listNumberedItems.append(numbered_list)   
listNumberedItems

[[1, 2],
 [3],
 [3, 5],
 [4, 3],
 [4],
 [1, 5, 2],
 [2, 3, 1],
 [4, 2, 1],
 [2, 3, 1, 5],
 [4, 5, 2, 1],
 [1, 3],
 [5, 1],
 [5],
 [5, 3],
 [4, 3, 1]]

In [42]:
data=raw_data.copy()
data['items']=listNumberedItems
data

,tid,items
0,1,"[1, 2]"
1,2,[3]
2,3,"[3, 5]"
3,4,"[4, 3]"
4,5,[4]
5,6,"[1, 5, 2]"
6,7,"[2, 3, 1]"
7,8,"[4, 2, 1]"
8,9,"[2, 3, 1, 5]"
9,10,"[4, 5, 2, 1]"


# Create First C

In [43]:
list_C, list_L = [], []

In [44]:
def countItemSet(itemSet):
    itemSet['Support'] = 0
    for row in data['items']:
        for index, item in enumerate(itemSet['ItemSet']):
            if len(item) > len(row):
                continue
            if all(elem in row for elem in item):
                itemSet.loc[index, 'Support'] += 1
    return itemSet

In [45]:
item_set = [item_no]
item_set = list(map(list, zip(*item_set)))
list_C.append(pd.DataFrame({'ItemSet':item_set}))

list_C[0]

,ItemSet
0,[1]
1,[2]
2,[3]
3,[4]
4,[5]


In [46]:
list_C[0] = countItemSet(list_C[0])
list_C[0]

,ItemSet,Support
0,[1],9
1,[2],6
2,[3],8
3,[4],5
4,[5],7


In [47]:
len(list_C[0])

5

# Create Secondary ItemSet

In [48]:
minimumSupport = int(input())

3


In [49]:
def removeLowestItemSet(C):
    global minimumSupport
    # minimumSupport = C['Support'].min()
    while C['Support'].min() < minimumSupport:
        C = C.drop(C['Support'].idxmin())
    return C

In [50]:
list_C[-1]

,ItemSet,Support
0,[1],9
1,[2],6
2,[3],8
3,[4],5
4,[5],7


In [51]:
def findNextC(L):
    itemSetBefore = L['ItemSet']
    itemSet = []
    for i in range(len(L['ItemSet']) - 1):
        item = L['ItemSet'].iloc[i]
        for j in range(i+1, len(L['ItemSet'])):
            row = item.copy()
            for k in range(len(item)):
                if item[k] == L['ItemSet'].iloc[j][k] :
                    continue
                elif k == len(item) - 1:
                    if item[k] < L['ItemSet'].iloc[j][k]:
                        row.append(L['ItemSet'].iloc[j][k])
                        #print("added row ", row)
                else:
                    break
            if len(row) == len(item) + 1:
                itemSet.append(row)
    newItemSet = pd.DataFrame({'ItemSet':itemSet})
    return newItemSet

In [52]:
def validate_item_set(C):
    validated_item_set = []
    for row in C['ItemSet']:
        #print(row)
        for numberedItem in listNumberedItems:
            if all(elem in numberedItem for elem in row):   
                validated_item_set.append(row)
                break
    newItemSet = pd.DataFrame({'ItemSet':validated_item_set})
    return newItemSet

In [53]:
list_C[0]

,ItemSet,Support
0,[1],9
1,[2],6
2,[3],8
3,[4],5
4,[5],7


In [54]:
while True:
    new_L = removeLowestItemSet(list_C[-1])
    list_L.append(new_L)
    if len(new_L) <= 1:
        break
    #print(list_L[-1])
    
    new_C = findNextC(list_L[-1])
    new_C = validate_item_set(new_C)
    new_C = countItemSet(new_C)
    #print(new_C)
    list_C.append(new_C)

In [55]:
for row in list_C:
    print(row)

  ItemSet  Support
0     [1]        9
1     [2]        6
2     [3]        8
3     [4]        5
4     [5]        7
  ItemSet  Support
0  [1, 2]        6
1  [1, 3]        4
2  [1, 4]        3
3  [1, 5]        4
4  [2, 3]        2
5  [2, 4]        2
6  [2, 5]        3
7  [3, 4]        2
8  [3, 5]        3
9  [4, 5]        1
     ItemSet  Support
0  [1, 2, 3]        2
1  [1, 2, 4]        2
2  [1, 2, 5]        3
3  [1, 3, 4]        1
4  [1, 3, 5]        1
5  [1, 4, 5]        1


In [56]:
for row in list_L:
    print(row)

  ItemSet  Support
0     [1]        9
1     [2]        6
2     [3]        8
3     [4]        5
4     [5]        7
  ItemSet  Support
0  [1, 2]        6
1  [1, 3]        4
2  [1, 4]        3
3  [1, 5]        4
6  [2, 5]        3
8  [3, 5]        3
     ItemSet  Support
2  [1, 2, 5]        3
